In [13]:
import os
import time
import logging
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler


In [14]:
# Configuration du logger pour écrire dans un fichier log.txt
log_file = "log.txt"
logging.basicConfig(
    filename=log_file,  # Enregistre les logs dans ce fichier
    level=logging.INFO,  # Niveau de logging
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"  # Mode "append" pour conserver les anciens logs
)

In [19]:
from pythonProject.get_from_file import read_excel
from pythonProject.get_from_api import fetch_data
import json
from actions import comparer_json
from actions import send_json_data

excel_file = "data.xlsx"
sheet_name = "Sheet1"
api_url = "https://essential-orelie-devpro13-5241d920.koyeb.app/kapi" 
class ExcelFileHandler(FileSystemEventHandler):
    """Classe pour gérer les événements de modification du fichier Excel."""

    def on_modified(self, event):
        if event.src_path == os.path.abspath(excel_file):
            logging.info(f"Le fichier {excel_file} a été modifié, traitement en cours...")

            try:
                # Récupération des données depuis l'API
                api_data = fetch_data(api_url)  # Assurez-vous que cette fonction existe
                with open("content_data.json", "w") as api_file:
                    json.dump(api_data, api_file)

                # Récupération des données depuis le fichier Excel
                excel_data = read_excel(excel_file, sheet_name)  # Assurez-vous que cette fonction existe
                with open("excel_content_data.json", "w") as excel_file_data:
                    json.dump(excel_data, excel_file_data)

                # Comparaison des données
                comparer_json("content_data.json", "excel_content_data.json", "pending_data.json")

                # Vérification si des données doivent être envoyées
                with open("pending_data.json") as f:
                    pending_data = json.load(f)
                    if pending_data:
                        logging.info("Nouvelles données détectées, envoi en cours...")
                        api_endpoint = "https://essential-orelie-devpro13-5241d920.koyeb.app/kapi/ask-kapi"
                        send_json_data(api_endpoint, "pending_data.json")
                    else:
                        logging.info("Aucune nouvelle donnée à envoyer.")

            except Exception as e:
                logging.error(f"Erreur lors du traitement du fichier : {e}")

# Initialisation de l'observateur de fichiers
event_handler = ExcelFileHandler()
observer = Observer()
observer.schedule(event_handler, path=os.path.dirname(os.path.abspath(excel_file)), recursive=False)

logging.info(f"Surveillance du fichier {excel_file} en cours...")
observer.start()

try:
    while True:
        time.sleep(1)  # Garde le programme actif
except KeyboardInterrupt:
    logging.info("Arrêt du programme...")
    observer.stop()
    observer.join()

❌ Erreur : Fichier non trouvé [Errno 2] No such file or directory: 'api_data.json'
❌ Erreur lors de l'envoi des données : [Errno 2] No such file or directory: 'pending_data.json'
❌ Erreur : Fichier non trouvé [Errno 2] No such file or directory: 'api_data.json'
❌ Erreur lors de l'envoi des données : [Errno 2] No such file or directory: 'pending_data.json'
✅ 1 éléments enregistrés dans pending_data.json
✅ Données envoyées avec succès : {'content': {'id': '67003dbddf91d3d20b5b88d2', 'users': 'Mr Ali', 'typeKapi': 'kapi aide', 'montant': 40, 'delais': 10, 'creationKapi': '20 déc. 2024'}, 'status': 'attente', '_id': '67d3f7c7d91209cdef8ae55f', 'createdAt': '2025-03-14T09:32:55.081Z', 'updatedAt': '2025-03-14T09:32:55.081Z', '__v': 0}
✅ Aucun nouvel élément trouvé. Aucune action requise.
✅ Données envoyées avec succès : {'content': {'id': '67003dbddf91d3d20b5b88d2', 'users': 'Mr Ali', 'typeKapi': 'kapi aide', 'montant': 40, 'delais': 10, 'creationKapi': '20 déc. 2024'}, 'status': 'attente',